<a href="https://colab.research.google.com/github/PARULCHUTANIPC/parul/blob/keras/vgg16mnist%20try.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
from tensorflow import keras

In [51]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

Using TensorFlow backend.


#*Preparing the VGG Model

In [0]:
vgg16 = keras.applications.vgg16.VGG16()

In [4]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [5]:
# load VGG16, ensuring the head FC layer sets are left off, while at
# the same time adjusting the size of the input image tensor to the
# network
baseModel = VGG16(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(128, 128, 3)))
# show a summary of the base model
print("[INFO] summary for base model...")
print(baseModel.summary())

[INFO] summary for base model...
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)  

In [0]:
# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
'''
headModel = AveragePooling2D(pool_size=(4, 4))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
'''
headModel = Dense(10, activation="softmax")(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)
# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

In [7]:
# compile our model (this needs to be done after our setting our
# layers to being non-trainable)
print("[INFO] compiling model...")
opt = tf.keras.optimizers.Adam(lr=1e-4)
model.compile(loss="binary_crossentropy", optimizer=opt,
	metrics=["accuracy"])

[INFO] compiling model...


#Preparing the Dataset

In [0]:
import pandas as pd

In [19]:
#testt= pd.read_csv('/content/drive/My Drive/Colab Notebooks/DATASETS/fashion-mnist_test.csv')
df = pd.read_csv("/content/drive/My Drive/Colab Notebooks/DATASETS/complete fmnist data.csv")
df.shape

(70000, 785)

In [45]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.14285)
train.shape, test.shape

((60000, 785), (10000, 785))

In [21]:
batch_size = 60
epochs = 10

# Initiate the train and test generators
train_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_dataframe(dataframe=train,
                                                    directory=None,
                                                    x_col='train_images',
                                                    y_col='train_labels',
                                                    has_ext=True,
                                                    target_size = (img_height,
                                                                   img_width),
                                                    batch_size = batch_size, 
                                                    class_mode = 'categorical',
                                                    color_mode = 'rgb')

validation_generator = test_datagen.flow_from_dataframe(dataframe=val,
                                                        directory=None,
                                                        x_col='test_images',
                                                        y_col='test_labels',
                                                        has_ext=True,
                                                        target_size = (img_height, 
                                                                       img_width),
                                                        batch_size = batch_size, 
                                                        class_mode = 'categorical',
                                                        color_mode = 'rgb')


NameError: ignored

In [0]:
fmnist = tf.keras.datasets.fashion_mnist

In [0]:
(train_images, train_labels),(test_images, test_labels)= fmnist.load_data()

In [54]:
train_images.shape, train_labels.shape, test_images.shape, test_labels.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [55]:
import cv2
import numpy as np
img = cv2.imread(train_images[0])
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img2 = np.zeros_like(img)
img2[:,:,0] = gray
img2[:,:,1] = gray
img2[:,:,2] = gray
cv2.imwrite('t1.jpg', img2)

SystemError: ignored

In [20]:
# Convert the images into 3 channels
train_images=np.dstack([train_images], 3)
test_images=np.dstack([test_images]*3)
train_images.shape,test_images.shape

TypeError: ignored

In [46]:
train_images[0].reshape()

(128, 128, 1)

## Resizing the images

Resizing to make it tensor

In [40]:
# * unpack the tuple
train_images = train_images.reshape(train_images.shape[0], *(28, 28, 1))
test_images = test_images.reshape(test_images.shape[0], *(28, 28, 1))
#validating_images = validating_images.reshape(validating_images.shape[0], *(28, 28, 1))
training_images.shape, test_images1.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

Rezining according to input required by VGG16 Model

In [42]:
from keras.preprocessing.image import img_to_array, array_to_img
train_images = np.asarray([img_to_array(array_to_img(im, scale=False).resize((128,128))) for im in train_images])
test_images = np.asarray([img_to_array(array_to_img(im, scale=False).resize((128,128))) for im in test_images])
#train_x = preprocess_input(x)
train_images.shape, test_images.shape

((60000, 128, 128, 1), (10000, 128, 128, 1))

## **Training the Model**

In [43]:
model.fit(x= train_images, y = train_labels, batch_size=100, epochs=10, verbose=2, validation_split=0.2)

ValueError: ignored

In [0]:
#fmnist = keras.data.datasets.fashion_mnist()
mnist = tf.keras.datasets.mnist

In [0]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [38]:
len(train_images),  len(train_labels), len(test_images), len(test_labels)

(60000, 60000, 10000, 10000)

In [39]:
train_images.shape, test_images.shape

((60000, 28, 28), (10000, 28, 28))

In [40]:
train_images[0].shape

(28, 28)

In [0]:
train_images0 = train_images[0].resize((224,224,3))

ValueError: ignored

In [0]:
catvsdog = ("/content/cat.3.jpg")

In [0]:
dataset = ("/content/drive/My Drive/Colab Notebooks/DATASETS/DOGS VS CATS/dogs-vs-cats.zip")

In [0]:
dataset

'/content/drive/My Drive/Colab Notebooks/DATASETS/DOGS VS CATS/dogs-vs-cats.zip'

In [0]:
from zipfile import ZipFile
file_name = "/content/drive/My Drive/Colab Notebooks/DATASETS/DOGS VS CATS/dogs-vs-cats.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print("Done")

Done


In [0]:
from zipfile import ZipFile
file_name = "/content/train.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print("Done")

Done


In [0]:
from zipfile import ZipFile
file_name = "/content/test1.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print("Done")

Done


In [0]:
test_data = ("/content/test1")
IMAGE_SIZE=224

In [0]:
test_data[0]

'/'

'/content/test1'